# TensorBoard

After Training the model, run 

    tensorboard --logdir=path/to/log-directory
    
Tensorboard provides a good visualization tool for all the variables you like and works on a browser.

In [1]:
import tensorflow as tf
import input_data

In [2]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

('Extracting', 'MNIST_data/train-images-idx3-ubyte.gz')
('Extracting', 'MNIST_data/train-labels-idx1-ubyte.gz')
('Extracting', 'MNIST_data/t10k-images-idx3-ubyte.gz')
('Extracting', 'MNIST_data/t10k-labels-idx1-ubyte.gz')


/usr/lib/python2.7/gzip.py:268: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:42: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


In [3]:
X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [4]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

(0, 0.92980003)
(1, 0.96600008)
(2, 0.97030014)
(3, 0.97340012)
(4, 0.9769001)
(5, 0.97820014)
(6, 0.98100019)
(7, 0.97890013)
(8, 0.97900015)
(9, 0.97850013)
(10, 0.97970009)
(11, 0.98210013)
(12, 0.98150009)
(13, 0.98180008)
(14, 0.9805001)
(15, 0.9823001)
(16, 0.98210013)
(17, 0.98240012)
(18, 0.98210025)
(19, 0.98210007)
(20, 0.98340011)
(21, 0.98350012)
(22, 0.98400009)
(23, 0.98260015)
(24, 0.98320013)
(25, 0.98180014)
(26, 0.98420012)
(27, 0.98390013)
(28, 0.9841001)
(29, 0.98250014)
(30, 0.98320013)
(31, 0.98200011)
(32, 0.98290014)
(33, 0.9836002)
(34, 0.9823001)
(35, 0.98340011)
(36, 0.98450017)
(37, 0.98350012)
(38, 0.98270011)
(39, 0.98390013)
(40, 0.98350012)
(41, 0.98450011)
(42, 0.98330015)
(43, 0.98360014)
(44, 0.98210019)
(45, 0.98360008)
(46, 0.98490018)
(47, 0.98360008)
(48, 0.98430008)
(49, 0.98290014)
(50, 0.98360014)
(51, 0.98310018)
(52, 0.98350012)
(53, 0.98350012)
(54, 0.98260021)
(55, 0.98340011)
(56, 0.98320007)
(57, 0.98460013)
(58, 0.98540008)
(59, 0.984500